# Generate Glue Catalog database and sample

### Here, we are going to generate the Glue Catalog database and tables so that we can experiment with Langchain.

In [1]:
import awswrangler as wr
import pandas as pd

In [2]:
# Load your Titanic dataset
df = pd.read_csv('titanic.csv')

# Split the dataframe into 2 tables for testing sake
passenger_info_df = df[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket']]
survival_info_df = df[['PassengerId', 'Survived']]

In [3]:
# Define the database name
database_name = "titanic_database"
passenger_table_name = "passenger_info"
survival_table_name = "survival_info"
s3_bucket = "metadocs-ai-blog-example"

In [4]:
# Create the database (if it doesn't exist)
wr.catalog.create_database(name=database_name, exist_ok=True)

In [5]:
# Define the S3 path where you'll store your tables
s3_path_passenger_info = f"s3://{s3_bucket}/table/{passenger_table_name}/"
s3_path_survival_info = f"s3://{s3_bucket}/table/{survival_table_name}/"

In [6]:
# Save passenger_info_df to S3 and create a table in the Glue Catalog
wr.s3.to_parquet(
    df=passenger_info_df,
    path=s3_path_passenger_info,
    dataset=True,
    database=database_name,
    table=passenger_table_name,
)

# Save survival_info_df to S3 and create a table in the Glue Catalog
wr.s3.to_parquet(
    df=survival_info_df,
    path=s3_path_survival_info,
    dataset=True,
    database=database_name,
    table=survival_table_name,
)

{'paths': ['s3://metadocs-ai-blog-example/table/survival_info/470729ec69dc4008a98c7b33f09e48eb.snappy.parquet'],
 'partitions_values': {}}

# Use Langchain to query the data

In [4]:
from langchain_community.document_loaders.athena import AthenaLoader

In [5]:
s3_output_path = f"s3://{s3_bucket}/query_results/"
query = f"SELECT * FROM {passenger_table_name} limit 20"

loader = AthenaLoader(
    query=query,
    database=database_name,
    s3_output_uri=s3_output_path,
    profile_name="default"
)

documents = loader.load()
print(documents)

[Document(page_content='passengerid: 1\npclass: 3\nname: Braund, Mr. Owen Harris\nsex: male\nage: 22.0\nsibsp: 1\nparch: 0\nticket: A/5 21171'), Document(page_content='passengerid: 2\npclass: 1\nname: Cumings, Mrs. John Bradley (Florence Briggs Thayer)\nsex: female\nage: 38.0\nsibsp: 1\nparch: 0\nticket: PC 17599'), Document(page_content='passengerid: 3\npclass: 3\nname: Heikkinen, Miss. Laina\nsex: female\nage: 26.0\nsibsp: 0\nparch: 0\nticket: STON/O2. 3101282'), Document(page_content='passengerid: 4\npclass: 1\nname: Futrelle, Mrs. Jacques Heath (Lily May Peel)\nsex: female\nage: 35.0\nsibsp: 1\nparch: 0\nticket: 113803'), Document(page_content='passengerid: 5\npclass: 3\nname: Allen, Mr. William Henry\nsex: male\nage: 35.0\nsibsp: 0\nparch: 0\nticket: 373450'), Document(page_content='passengerid: 6\npclass: 3\nname: Moran, Mr. James\nsex: male\nage: None\nsibsp: 0\nparch: 0\nticket: 330877'), Document(page_content='passengerid: 7\npclass: 1\nname: McCarthy, Mr. Timothy J\nsex: male

In [6]:
import pandas as pd

In [8]:
data = []

# Iterate over each document in the output
for doc in documents:
    passenger_data = {}
    lines = doc.page_content.split('\n')
    for line in lines:
        key, value = line.split(': ', 1)
        passenger_data[key] = value
    data.append(passenger_data)

df = pd.DataFrame(data)

In [9]:
df

,passengerid,pclass,name,sex,age,sibsp,parch,ticket
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450
5,6,3,"Moran, Mr. James",male,None,0,0,330877
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736
